In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2
from cv2 import imread, imwrite
import os
import glob
from tensorflow.image import psnr as psnr
from tensorflow.keras import datasets, layers, models
import matplotlib.image as mpimg
from matplotlib.pyplot import savefig
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.datasets.mnist import load_data
from numpy import reshape
import matplotlib.pyplot as plt
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()

In [2]:
path_airplane = 'natural_images_kaggle\\natural_images\\airplane\\*.jpg'
path_car = 'natural_images_kaggle\\natural_images\\car\\*.jpg'
path_cat = 'natural_images_kaggle\\natural_images\\cat\\*.jpg'
path_dog = 'natural_images_kaggle\\natural_images\\dog\\*.jpg'
path_flower = 'natural_images\\flower\\*.jpg'
path_fruit = 'natural_images\\fruit\\*.jpg'
path_motorbike = 'natural_images_kaggle\\natural_images\\motorbike\\*.jpg'
path_person = 'natural_images_kaggle\\natural_images\\person\\*.jpg'

#paths_list = [path_airplane, path_car, path_cat, path_dog, path_flower, path_fruit,  path_motorbike, path_person]

In [4]:
paths_list = [path_airplane, path_car, path_cat, path_dog, path_flower, path_fruit, path_motorbike, path_person]

acum = 1
for path in paths_list:
    filenames = glob.glob(path)
    for filename in filenames:
        image = cv2.imread(filename)
        gray_img = cv2.imread(filename, 0)
        image = cv2.resize(image, (128, 128))
        gray_img = cv2.resize(gray_img, (128, 128))
        cv2.imwrite("gray_images/gray_" +str(acum) +".jpg", gray_img)
        cv2.imwrite("color_images/color_" +str(acum) +".jpg", image)
        acum += 1

In [5]:
dataset = []

num_images = 1843

#Read all grayscale images and append into numpy list
for i in range(1, num_images+1):
    img = cv2.imread("gray_images/gray_" +str(i) +".jpg", 0)
    dataset.append(np.array(img))

dataset_source = np.asarray(dataset)
print(dataset_source.shape)

dataset_tar = []

#Read all color images and append into numpy list
for i in range(1, num_images+1):
    img = cv2.imread("color_images/color_" +str(i) +".jpg")    
    dataset_tar.append(np.array(img))

dataset_target = np.asarray(dataset_tar)
print(dataset_target.shape) 

(1843, 128, 128)
(1843, 128, 128, 3)


In [7]:
dataset_target = dataset_target.astype('float32') /255.0
dataset_target = dataset_target.astype('float32') /255.0
print(dataset_source.shape, dataset_target.shape) 

dataset_source = dataset_source.reshape((len(dataset_source), 128, 128, 1))
dataset_target = dataset_target.reshape((len(dataset_target), 128, 128, 3))
print(dataset_source.shape, dataset_target.shape) 

(1843, 128, 128) (1843, 128, 128, 3)
(1843, 128, 128, 1) (1843, 128, 128, 3)


In [17]:
input_img = Input(shape=(128, 128, 1))

enc_conv1 = Conv2D(12, (3, 3), activation='relu', padding='same')(input_img)
enc_pool1 = MaxPooling2D((2, 2), padding='same')(enc_conv1)
enc_conv2 = Conv2D(8, (4, 4), activation='relu', padding='same')(enc_conv1)
enc_ouput = MaxPooling2D((4, 4), padding='same')(enc_conv2)
    
# Decoder
dec_conv2 = Conv2D(8, (4, 4), activation='relu', padding='same')(enc_conv2)
dec_upsample2 = UpSampling2D((4, 4))(dec_conv2)
dec_conv3 = Conv2D(12, (3, 3), activation='relu', padding='same')(dec_conv2)
dec_upsample3 = UpSampling2D((2, 2))(dec_conv3)
#dec = tf.image.resize_nearest_neighbor(dec_output, tf.constant([129, 129]))
dec_output = Conv2D(3, (3, 3), activation=None, padding='same')(enc_conv1)

autoencoder = Model(input_img, dec_output)
autoencoder.compile(optimizer='rmsprop', loss='mae')
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 128, 128, 12)      120       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 128, 128, 3)       327       
Total params: 447
Trainable params: 447
Non-trainable params: 0
_________________________________________________________________


In [18]:
autoencoder.fit(dataset_source, dataset_target, epochs=3, batch_size=32, shuffle=True)

Epoch 1/3
58/58 [==============================] - 8s 140ms/step - loss: 13.3720
Epoch 2/3
58/58 [==============================] - 9s 149ms/step - loss: 3.2723
Epoch 3/3
58/58 [==============================] - 8s 145ms/step - loss: 2.5523


In [19]:
dataset_target[0].shape, dataset_source[0].shape

((128, 128, 3), (128, 128, 1))

In [20]:
predict = autoencoder.predict(dataset_target[0])
plt.imshow(predict)

ValueError: in user code:

    C:\Users\Gman\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    C:\Users\Gman\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Gman\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Gman\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Gman\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Gman\A